# 畳み込み符号

In [2]:
import numpy as np
from scipy.sparse.csgraph import shortest_path
import itertools

In [26]:
class MyCircuit:
    """
    回路
    """

    def __init__(self, status_num=0):
        # シフトレジスタ
        self.sr = [[0], [0, 0]]
        # 生成系列
        self.g = [
            [[1, 1], [0, 0, 0]],
            [[0, 1], [0, 1, 1]],
            [[1, 0], [1, 0, 1]]
        ]
        
        self.sr = [[0, 0, 0]]
        self.g = [
            [[1, 0, 1, 1]],
            [[1, 1, 1, 1]]
        ]
        
        # メモリ数
        self.v = sum(len(i) for i in self.sr)
        # 同時入力数??
        self.u_len = len(self.sr)
        self._set_sr(status_num)
        self.status_num = self._calc_status_num()
    
    def transition(self, u_list):
        """
        状態遷移
        """
        w_list = []
        for g_list in self.g:
            w = 0
            for i, u in enumerate(u_list):
                sr_vec = np.array([u] + self.sr[i])
                g_vec = np.array(g_list[i])
                w += np.dot(sr_vec, g_vec)
            w_list.append(w % 2)
        for i in range(len(self.sr)):
            self.sr[i] = [u_list[i]] + self.sr[i][:-1]
        self.status_num = self._calc_status_num()
        return w_list
    
    def _calc_status_num(self):
        """
        レジスタの中身から状態番号計算
        """
        return int("".join([str(i) for i in itertools.chain.from_iterable(self.sr)]), 2)
    
    def _set_sr(self, status_num):
        """
        シフトレジスタの中身を指定した状態に対応させる
        """
        idx = 0
        st_bin_str = format(status_num, f"0{self.v}b")
        for i in range(len(self.sr)):
            for j in range(len(self.sr[i])):
                self.sr[i][j] = int(st_bin_str[idx])
                idx += 1
    
    def __str__(self):
        moji = f"S_{self.status_num}\n"
        for i, j in enumerate(self.sr):
            moji += f"sr{i}: {j}\n"
        return moji

def create_state_transition_dict():
    """
    状態遷移の辞書を作成
    """
    c = MyCircuit(1)
    # 全状態
    st_tr_dic = {i : {} for i in range(1 << c.v)}
    # 全入力パターン
    act_list = [[int(j) for j in format(i, f"0{c.u_len}b")] for i in range(1 << c.u_len)]
    for st in st_tr_dic.keys():
        for u_list in act_list:
            c = MyCircuit(st)
            w_list = c.transition(u_list)
            st_tr_dic[st][c.status_num] = [u_list, w_list]
    return st_tr_dic

def to_adjacency_matrix(st_tr_dic: dict):
    """
    状態遷移の辞書から隣接行列作成
    重み付き有効グラフとする
    重みは出力のハミング重み
    """
    m = np.zeros([len(st_tr_dic)] * 2)
    for src, child_dic in st_tr_dic.items():
        for dst, uw in child_dic.items():
            weight = sum(uw[1])
            # 重み0は辺が無いと判定されるため微小な値を入れる
            if weight == 0:
                weight = 1e-5
            m[src][dst] = weight
    return m

def get_path(src, dst, p):
    sp = []
    p_row = p[src]
    i = dst
    while i != src and i >= 0:
        sp.append(i)
        i = p_row[i]
    if i < 0:
        sp = []
    else:
        sp.append(i)
    return sp[::-1]

In [28]:
incidence_dict = create_state_transition_dict()
for k, v in incidence_dict.items():
    print(k, v)
m = to_adjacency_matrix(incidence_dict)
# print(m)
saitan_w, saitan_p = shortest_path(m, method="D", return_predecessors=True)
saitan_w = saitan_w.astype("u1")
print(saitan_w)
print(saitan_p)
# print(get_path(2, 6, saitan_p))

0 {0: [[0], [0, 0]], 4: [[1], [1, 1]]}
1 {0: [[0], [1, 1]], 4: [[1], [0, 0]]}
2 {1: [[0], [1, 1]], 5: [[1], [0, 0]]}
3 {1: [[0], [0, 0]], 5: [[1], [1, 1]]}
4 {2: [[0], [0, 1]], 6: [[1], [1, 0]]}
5 {2: [[0], [1, 0]], 6: [[1], [0, 1]]}
6 {3: [[0], [1, 0]], 7: [[1], [0, 1]]}
7 {3: [[0], [0, 1]], 7: [[1], [1, 0]]}
[[0 4 3 4 2 3 3 4]
 [2 0 1 2 0 1 1 2]
 [4 2 0 2 2 0 1 2]
 [2 0 1 0 0 1 1 2]
 [4 2 1 2 0 1 1 2]
 [4 2 1 2 2 0 1 2]
 [3 1 2 1 1 2 0 1]
 [3 1 2 1 1 2 2 0]]
[[-9999     3     4     6     0     2     4     6]
 [    1 -9999     4     6     1     2     4     6]
 [    1     2 -9999     6     1     2     5     6]
 [    1     3     4 -9999     1     2     4     6]
 [    1     3     4     6 -9999     2     4     6]
 [    1     3     5     6     1 -9999     5     6]
 [    1     3     4     6     1     2 -9999     6]
 [    1     3     4     7     1     2     4 -9999]]


In [33]:
# 最短自由距離を計算
# S_0からS_iの間を行って帰ってくるまでの最短距離を計算
# i>0について繰り返し、最小のものが最短自由距離。多分
d_free = float("inf")
d_free_path = []

for i in range(1, saitan_w.shape[0]):
    cycle_len_min_i = saitan_w[0][i] + saitan_w[i][0]
    cycle_path_min_i = get_path(0, i, saitan_p) + get_path(i, 0, saitan_p)[1:]
    print(f"i={i}, length_min={cycle_len_min_i}")
    print(f"path={cycle_path_min_i}")
    if cycle_len_min_i < d_free:
        d_free = cycle_len_min_i
        d_free_path = cycle_path_min_i
print(f"最小自由距離: {d_free}, 経路: {d_free_path}")

i=1, length_min=6
path=[0, 4, 6, 3, 1, 0]
i=2, length_min=7
path=[0, 4, 2, 1, 0]
i=3, length_min=6
path=[0, 4, 6, 3, 1, 0]
i=4, length_min=6
path=[0, 4, 6, 3, 1, 0]
i=5, length_min=7
path=[0, 4, 2, 5, 6, 3, 1, 0]
i=6, length_min=6
path=[0, 4, 6, 3, 1, 0]
i=7, length_min=7
path=[0, 4, 6, 7, 3, 1, 0]
最小自由距離: 6, 経路: [0, 4, 6, 3, 1, 0]
